In [ ]:
"""
core/service.py - The Core LLM Service Orchestrator.

This module defines the `LLMService` class, which is the central hub for
the chatbot's logic. It orchestrates all the core components to process a user
query, generate a response, and log the entire process to MLflow.
"""

import time
import logging
import mlflow
from typing import List, Optional

from langchain_anthropic import ChatAnthropic

from core.rag import RAGPipeline
from core.router import HeuristicRouter
from core.adapter import ControlHintAdapter
from core.guardrail import Guardrail
from config import CONFIG

# Set up logging for this module
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class LLMService:
    """
    Orchestrates the LLM chatbot's core functionality.

    This service is responsible for integrating the Guardrail, RAG, Adapter, and Router
    components. It handles the end-to-end process of answering a user's query and
    logging the results to MLflow for observability.
    """
    def __init__(self, rag: RAGPipeline, adapter: ControlHintAdapter,
                 router: HeuristicRouter, guardrail: Guardrail):
        """
        Initializes the LLMService with all necessary components.
        """
        self.rag = rag
        self.adapter = adapter
        self.router = router
        self.guardrail = guardrail

    async def answer(self, query: str) -> str:
        """
        Processes a user query and returns a response from the LLM.

        This method performs the following steps:
        1. Checks the query with the guardrail.
        2. Selects an LLM model using the router.
        3. Retrieves relevant context from the RAG pipeline.
        4. Adapts the prompt with the control hint.
        5. Invokes the selected LLM and logs the result to MLflow.
        6. Checks the LLM's response with the guardrail.
        7. Returns the final response.

        Args:
            query: The user's question.

        Returns:
            The generated response string.
        """
        # 1. Guardrail Check (on user query)
        if not self.guardrail.check(query):
            logger.warning(f"Query flagged by guardrail: '{query}'")
            return self.guardrail.safe_response()

        # 2. Select model
        model = self.router.select_model(query)
        client = ChatAnthropic(model=model, api_key=CONFIG.anthropic_api_key)

        # 3. Retrieve context
        context = self.rag.build_context(query)

        # 4. Adapt prompt
        prompt = self.adapter.adapt(f"Context:\n{context}\n\nQuestion: {query}")

        start_time = time.time()
        with mlflow.start_run(nested=True, run_name="llm_inference"):
            try:
                # 5. Invoke LLM and log
                response = await client.ainvoke(prompt)
                latency = time.time() - start_time
                response_content = response.content

                # 6. Guardrail Check (on LLM response)
                if not self.guardrail.check(response_content):
                    logger.warning("LLM response flagged by guardrail.")
                    mlflow.log_param("moderation_status", "flagged")
                    return self.guardrail.safe_response()

                # Log all relevant information to MLflow
                mlflow.log_param("model", model)
                mlflow.log_param("query", query)
                mlflow.log_metric("latency_ms", latency * 1000)
                mlflow.log_text(response_content, "response.txt")
                mlflow.log_param("moderation_status", "passed")

                logger.info(f"Response generated in {latency:.2f}s using {model}.")
                return response_content
            except Exception as e:
                mlflow.log_param("error", str(e))
                logger.error(f"Failed to get response from LLM: {e}", exc_info=True)
                return "[Error: An internal service error occurred.]"